# My Simple ID3

## Imports

In [1]:
import pandas as pd
import numpy as np
import pprint

## Load and Prepare Data

In [2]:
full_car_data = pd.read_csv("https://raw.githubusercontent.com/AdrianL-Work/MLF-TLS-UTS-2021/main/data/cars.csv")

In [3]:
full_car_data

,buying,maint,doors,persons,lug_boot,safety,quality
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [4]:
full_car_data.quality.unique()

array(['unacc', 'acc', 'vgood', 'good'], dtype=object)

In [5]:
#split into train and test.
training_car_data = full_car_data.sample(frac=0.70,random_state=200) #adjust the fraction as needed to split data
test_car_data = full_car_data.drop(training_car_data.index)

In [6]:
test_car_data

,buying,maint,doors,persons,lug_boot,safety,quality
0,vhigh,vhigh,2,2,small,low,unacc
2,vhigh,vhigh,2,2,small,high,unacc
10,vhigh,vhigh,2,4,small,med,unacc
15,vhigh,vhigh,2,4,big,low,unacc
17,vhigh,vhigh,2,4,big,high,unacc
...,...,...,...,...,...,...,...
1700,low,low,4,more,big,high,vgood
1705,low,low,5more,2,med,med,unacc
1718,low,low,5more,4,big,high,vgood
1719,low,low,5more,more,small,low,unacc


In [7]:
G_CATEGORY = 'quality'

## ID3

In [8]:
"""
Intended Structure
 - Calculate Entropy of the quality
 - calculate Entropy of all the set
 - calculate information gain
     - determine highest info gain
 - nodes
 - build the decision tree
 
 - predict function
 
 -main as controller for a big for loop.
"""

'\nIntended Structure\n - Calculate Entropy of the quality\n - calculate Entropy of all the set\n - calculate information gain\n     - determine highest info gain\n - nodes\n - build the decision tree\n \n - predict function\n \n -main as controller for a big for loop.\n'

In [9]:
def calc_key_entropy(df):
    '''
    Calculates the Entropy of the passed in key label.
    df - dataframe
    category - string
    '''
    entropy = 0
    values = df[G_CATEGORY].unique()
    for value in values:
        probability = df[G_CATEGORY].value_counts()[value]/len(df[G_CATEGORY])
        entropy += -probability*np.log2(probability)
    return entropy

In [10]:
calc_key_entropy(training_car_data)

1.2118818705419752

In [11]:
def calc_all_attribute_entropy(df, attr):
    '''
    Calculates Entropy of all attributes
    df - dataframe
    category - string
    attr - string
    '''  
    target_variables = df[G_CATEGORY].unique()  
    variables = df[attr].unique()
    eps = np.finfo(float).eps
    new_entropy = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
                num = len(df[attr][df[attr]==variable][df[G_CATEGORY] ==target_variable])
                den = len(df[attr][df[attr]==variable])
                probability = num/(den+eps)
                entropy += -probability*np.log2(probability+eps)
        new_probability = den/len(df)
        new_entropy += -new_probability*entropy
    return abs(new_entropy)

In [12]:
def highest_information_gain(df):
    '''
    Finds the attribute with the highest information gain.
    df - dataframe
    '''
    entropy_att = []
    info_gain = []
    for key in df.keys()[:-1]:
        info_gain.append(calc_key_entropy(df)-calc_all_attribute_entropy(df,key))
    highest_info_gain =  df.keys()[:-1][np.argmax(info_gain)]
    return highest_info_gain

In [13]:
def get_node(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

In [14]:
def build_id3_tree(df,tree=None): 
    '''
    Builds (Trains) the tree model.
    '''
    #Get attribute with maximum information gain as starting node
    node = highest_information_gain(df)
    attValue = np.unique(df[node])
     
    if tree is None:                    
        tree={}
        tree[node] = {}

    for value in attValue:
        subtable = get_node(df,node,value)
        clValue,counts = np.unique(subtable[G_CATEGORY],return_counts=True)                        
        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = build_id3_tree(subtable)
                   
    return tree

## Build Tree

In [15]:
tree = build_id3_tree(training_car_data)

In [16]:
pprint.pprint(tree)

{'safety': {'high': {'persons': {'2': 'unacc',
                                 '4': {'buying': {'high': {'maint': {'high': 'acc',
                                                                     'low': 'acc',
                                                                     'med': 'acc',
                                                                     'vhigh': 'unacc'}},
                                                  'low': {'maint': {'high': {'lug_boot': {'big': 'vgood',
                                                                                          'med': {'doors': {'2': 'acc',
                                                                                                            '3': 'acc',
                                                                                                            '4': 'vgood',
                                                                                                            '5more': 'vgood'}},
                  

In [17]:
def predict(inst,tree):
    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

## Testing and Evaluation

In [18]:
test_car_data

,buying,maint,doors,persons,lug_boot,safety,quality
0,vhigh,vhigh,2,2,small,low,unacc
2,vhigh,vhigh,2,2,small,high,unacc
10,vhigh,vhigh,2,4,small,med,unacc
15,vhigh,vhigh,2,4,big,low,unacc
17,vhigh,vhigh,2,4,big,high,unacc
...,...,...,...,...,...,...,...
1700,low,low,4,more,big,high,vgood
1705,low,low,5more,2,med,med,unacc
1718,low,low,5more,4,big,high,vgood
1719,low,low,5more,more,small,low,unacc


In [19]:
#single test to sanity check.
test_prediction = predict(test_car_data.iloc[250], tree)
test_prediction

'unacc'

In [20]:
def evaluate_model(data):
    tp = 0
    fp = 0
    fail = 0

    for index, row in data.iterrows():
        try:
            result = predict(row, tree)
        except:
            result = 0
            
        if result == row['quality']:
            tp += 1
        elif result !=0:
            fp += 1
        else:
            fail +=1
    return tp, fp, fail

In [21]:
tp, fp, fail = evaluate_model(test_car_data)

In [22]:
tp # true positive

456

In [23]:
fp # false positive

24

In [24]:
fail #error

38